### 2-2. 해외 기후 데이터 정제
- 단순하게 채소별로 묶여있는 기후 데이터를 머신러닝, LSTM 모델에 적합한 데이터 형식으로 변경

In [2]:
import pandas as pd
import numpy as np
import os

In [4]:
pd.read_csv('./data/climate_merged/배추_for.csv', index_col = 0)

,지점,지점명,일시,습도,16 방위 풍향,풍속,강수량,현지기압,해면기압,변화량 기압,이슬점 온도,기온,최고 기온,최저 기온
0,54527,텐진,2010-01-01 00:00,70.0,2.0,3.0,NaN,NaN,1017.2,0.3,-12.4,-7.9,NaN,NaN
1,54527,텐진,2010-01-01 03:00,62.2,2.0,2.0,NaN,NaN,1016.5,0.7,-11.2,-5.1,NaN,NaN
2,54527,텐진,2010-01-01 06:00,45.9,4.0,3.0,NaN,NaN,1014.3,2.2,-11.1,-0.9,NaN,-9.3
3,54527,텐진,2010-01-01 09:00,52.9,4.0,1.0,NaN,NaN,1015.1,0.8,-9.1,-0.7,NaN,NaN
4,54527,텐진,2010-01-01 12:00,78.2,9.0,4.0,NaN,NaN,1017.2,2.1,-4.5,-1.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5758,57083,정저우,2021-09-15 21:00,NaN,24.0,0.0,NaN,1000.0,1012.9,0.4,19.7,19.7,30.6,19.6
5759,57083,정저우,2021-09-16 00:00,63.1,9.0,1.0,NaN,1000.8,1013.6,0.8,17.3,24.8,30.6,19.6
5760,57083,정저우,2021-09-16 03:00,52.0,4.0,2.0,NaN,1000.8,1013.7,0.0,17.5,28.3,30.6,19.6
5761,57083,정저우,2021-09-16 06:00,51.0,3.0,2.0,NaN,998.9,1011.8,1.9,18.3,29.5,30.6,19.6


In [9]:
def max_or_zero(series):
    if series.notna().sum() == 0 :
        return 0
    else :
        return series[series.notna()].max()
    
def min_or_zero(series):
    if series.notna().sum() == 0 :
        return 0
    else :
        return series[series.notna()].min()

def mean_or_zero(series):
    if series.notna().sum() == 0 :
        return 0
    else :
        return series[series.notna()].mean()

In [10]:
convert_datas = {
    'Temp' : lambda x : x.mean(),
    'Rainfall' : max_or_zero,
    'WindSpeed' : max_or_zero,
    'Humidity' : lambda x : x.mean(),
    'SteamPressure' : lambda x : x.mean(),
    'LandAp' : lambda x : x.mean(),
    'SeaAp' : lambda x : x.mean(),
}

In [15]:
def convert_df(date, df) :
    morning = df.loc[date : date + ' 11:59:59']
    afternoon = df.loc[date + ' 12:00:00' : date]
    
    lines = {}
    
    for column, func in convert_datas.items():
        lines['m_' + column] = func(morning[column])
        
    for column, func in convert_datas.items():
        lines['a_' + column] = func(afternoon[column])
    
    lines['TempMin'] = min_or_zero(morning.Temp)
    lines['TempMax'] = max_or_zero(afternoon.Temp)
    
    if lines['TempMin'] == 0 :
        lines['TempMin'] = min_or_zero(morning.Temp)
    if lines['TempMax'] == 0 :
        lines['TempMax'] = max_or_zero(afternoon.Temp)
        
    lines['Freeze'] = (morning.Temp < morning.Dewpoint).sum() + (afternoon.Temp < afternoon.Dewpoint).sum()
    return lines

def concat_horizontal(df) : 
    location_list = df.Location.unique()
    merge_df_list = []
    for location in location_list :
        print(location, 'start')
        temp_df = df[df.Location == location].sort_values('Date')
        temp_list = []
        for date in dates :
            temp_list.append(pd.DataFrame(convert_df(date, temp_df), index = [date]))
        temp_df = pd.concat(temp_list)
        temp_df = temp_df.add_suffix(f'_{location}')
        merge_df_list.append(temp_df.ffill())
    return pd.concat(merge_df_list, axis = 1)

In [7]:
date_list = list(pd.date_range('2010-01-01', '2021-10-15'))
dates = []
for date in date_list:
    dates.append(str(date).split(' ')[0])

In [8]:
dates

['2010-01-01',
 '2010-01-02',
 '2010-01-03',
 '2010-01-04',
 '2010-01-05',
 '2010-01-06',
 '2010-01-07',
 '2010-01-08',
 '2010-01-09',
 '2010-01-10',
 '2010-01-11',
 '2010-01-12',
 '2010-01-13',
 '2010-01-14',
 '2010-01-15',
 '2010-01-16',
 '2010-01-17',
 '2010-01-18',
 '2010-01-19',
 '2010-01-20',
 '2010-01-21',
 '2010-01-22',
 '2010-01-23',
 '2010-01-24',
 '2010-01-25',
 '2010-01-26',
 '2010-01-27',
 '2010-01-28',
 '2010-01-29',
 '2010-01-30',
 '2010-01-31',
 '2010-02-01',
 '2010-02-02',
 '2010-02-03',
 '2010-02-04',
 '2010-02-05',
 '2010-02-06',
 '2010-02-07',
 '2010-02-08',
 '2010-02-09',
 '2010-02-10',
 '2010-02-11',
 '2010-02-12',
 '2010-02-13',
 '2010-02-14',
 '2010-02-15',
 '2010-02-16',
 '2010-02-17',
 '2010-02-18',
 '2010-02-19',
 '2010-02-20',
 '2010-02-21',
 '2010-02-22',
 '2010-02-23',
 '2010-02-24',
 '2010-02-25',
 '2010-02-26',
 '2010-02-27',
 '2010-02-28',
 '2010-03-01',
 '2010-03-02',
 '2010-03-03',
 '2010-03-04',
 '2010-03-05',
 '2010-03-06',
 '2010-03-07',
 '2010-03-

In [17]:
columns = ['지점', '일시', '습도', '풍속', '강수량', '현지기압', '해면기압',
           '변화량 기압', '이슬점 온도', '기온', '최고 기온', '최저 기온']

columns_rename = [
    'Location', 'Date', 'Humidity', 'WindSpeed', 'Rainfall', 'LandAp', 'SeaAp', 'SteamPressure', 'Dewpoint', 'Temp',
    'TempMin', 'TempMax']

In [18]:
path_lists = os.listdir('./data/climate_merged')
for path in path_lists:
    if 'na' in path :
        continue
    print(path, 'start')
    df = pd.read_csv(f'./data/climate_merged/{path}')
    df = df[columns]
    df.columns = columns_rename

    df['Date'] = pd.to_datetime(df.Date)
    df.set_index('Date', inplace = True)
    
    temp = concat_horizontal(df)
    print(temp.isnull().sum().sum())
    temp.to_csv(f'./data/climate_final/{path}', encoding = 'cp949')

고추_for.csv start
56778 start
57127 start
57816 start
58027 start
443
당근_for.csv start
58847 start
0
마늘_for.csv start
58027 start
0
무_for.csv start
52866 start
443
배추_for.csv start
54527 start
54823 start
57083 start
574
시금치_for.csv start
47662 start
0
쌀_for.csv start
72219 start
72259 start
72494 start
48820 start
48845 start
48914 start
54161 start
54342 start
57127 start
58238 start
58847 start
59287 start
59431 start
48378 start
48400 start
94693 start
94910 start
65730
양파_for.csv start
72698 start
72295 start
72793 start
47412 start
47772 start
47817 start
52533 start
54102 start
58847 start
54823 start
56778 start
50745 start
443


In [5]:
pd.read_csv('./data/climate_final/해외/배추_for.csv', index_col = 0)

,m_Temp_54527,m_Rainfall_54527,m_WindSpeed_54527,m_Humidity_54527,m_SteamPressure_54527,m_LandAp_54527,m_SeaAp_54527,m_TempMin_54527,m_TempMax_54527,a_Temp_54527,...,a_WindSpeed_57083,a_Humidity_57083,a_SteamPressure_57083,a_LandAp_57083,a_SeaAp_57083,a_TempMin_57083,a_TempMax_57083,TempMin_57083,TempMax_57083,Freeze_57083
2010-01-01,-3.650,0.0,3.0,57.750,1.000,NaN,1015.775,0.0,-9.3,-3.633333,...,2.0,47.300,1.950,1001.500,1016.300,17.0,0.0,-3.8,5.4,0
2010-01-02,-5.300,0.0,3.0,77.450,1.875,NaN,1026.875,0.0,-6.2,-6.400000,...,3.0,76.825,1.650,1010.800,1024.350,4.4,0.0,-2.3,-1.0,0
2010-01-03,-6.125,1.0,6.0,91.750,1.600,NaN,1020.875,0.0,-7.3,-13.566667,...,5.0,79.425,2.350,1006.350,1021.525,1.9,0.0,-3.6,-0.9,0
2010-01-04,-10.725,0.0,8.0,46.150,1.675,NaN,1029.475,0.0,-17.1,-13.800000,...,3.0,45.300,1.400,1018.300,1033.250,-0.1,0.0,-5.1,-3.1,0
2010-01-05,-12.700,0.0,4.0,44.800,1.125,NaN,1033.775,0.0,-18.1,-15.200000,...,2.0,35.525,0.625,1017.000,1031.825,-1.8,0.0,-6.8,-4.1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-11,21.600,0.0,0.0,84.975,0.825,1014.2,1014.800,0.0,0.0,20.633333,...,0.0,78.400,0.675,1000.775,1013.650,0.0,0.0,0.0,0.0,0
2021-10-12,21.600,0.0,0.0,84.975,0.825,1014.2,1014.800,0.0,0.0,20.633333,...,0.0,78.400,0.675,1000.775,1013.650,0.0,0.0,0.0,0.0,0
2021-10-13,21.600,0.0,0.0,84.975,0.825,1014.2,1014.800,0.0,0.0,20.633333,...,0.0,78.400,0.675,1000.775,1013.650,0.0,0.0,0.0,0.0,0
2021-10-14,21.600,0.0,0.0,84.975,0.825,1014.2,1014.800,0.0,0.0,20.633333,...,0.0,78.400,0.675,1000.775,1013.650,0.0,0.0,0.0,0.0,0
